<a href="https://colab.research.google.com/github/ElizavetaNosova/CompLing_homeworks/blob/master/KeywordExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
import nltk
nltk.download('stopwords')
stops = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [1]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 2.3MB/s 
     |████████████████████████████████| 7.1MB 7.7MB/s 


In [0]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

Идея 1: будем учитывать не только существительные, но и однокоренные им слова других частей речи.
Алгоритм:
1. Составляем словать "результат стеминга существительного - существительное", игнорируя слова других частей речи.
2. Делаем стеминг всего текста, оставляем только те единицы, которые есть в словаре.
3. Выявляем "ключевые результаты стеминга" с помощью графа
4. Ответам считаем соответствующие им существительные

Проверка на части данных

Сначала сделаем проверку на части данных, а потом пересчитаем хорошие решения для всех данных

In [5]:
from google.colab import files
uploaded = files.upload()

Saving ng_0.jsonlines to ng_0.jsonlines


In [0]:
test_data = pd.read_json('ng_0.jsonlines', lines=True)

In [0]:
test_data

,keywords,title,url,content,summary,content_norm,title_norm
0,"[яблоко, молодежь, молодежное яблоко]","""Молодежное ""Яблоко"": оппозиционная деятельнос...",http://www.ng.ru/ng_politics/2017-04-18/11_697...,"Многие интересуются, зачем нужна «Яблоку» моло...",,"[многие, интересоваться, нужный, яблоко, молод...","[молодёжный, яблоко, оппозиционный, деятельнос..."
1,"[газпром, газ]","""Газпрома"" на всех не хватит",http://www.ng.ru/economics/2008-04-03/1_gazpro...,Вчера «Газпром» снизил верхнюю планку прогноза...,,"[вчера, газпром, снизить, верхний, планка, про...","[газпром, хватить]"
2,"[франсуа рабле, сервантес, шекспир, конан дойл...",Бесконечная партия в четырехмерные шахматы,http://www.ng.ru/person/2018-03-22/10_927_vitk...,Долголетний труд Евгения Витковского на ниве п...,"Евгений Витковский о том, как Босх протягивает...","[долголетний, труд, евгений, витковский, нива,...","[бесконечный, партия, четырехмерный, шахматы]"
3,"[владивосток, суд, ким, футина, выборы, боевое...","Экс-депутат, осужденная за фальсификацию выбор...",http://www.ng.ru/regions/2018-01-10/100_vladiv...,В Ленинском районном суде продолжаются слушани...,Фигурантке уголовного дела о фальсификации выб...,"[ленинский, районный, суд, продолжаться, слуша...","[экс-депутат, осудить, фальсификация, выбор, о..."
4,"[новая москва, подмосковье, благоустройство, т...",Новая Москва останется территорией экологическ...,http://www.ng.ru/ng_stolitsa/2017-11-10/10_711...,В 2012 году российская столица резко увеличила...,Лучшие проекты благоустройства общественных пр...,"[2012, год, российский, столица, резко, увелич...","[новый, москва, остаться, территория, экологич..."
...,...,...,...,...,...,...,...
994,"[венгрия, виктор орбан, миграционный кризис, и...","Орбан призвал остановить ""исламскую экспансию""",http://www.ng.ru/world/2018-02-21/1_7177_hunga...,"В парламенте Венгрии обсудили законопроект, за...",Тема миграции остается оружием в руках правых ...,"[парламент, венгрия, обсудить, законопроект, з...","[орбан, призвать, остановить, исламский, экспа..."
995,"[парижское соглашение, климат, экология, парни...",Парижское соглашение: трудности перевода,http://www.ng.ru/energy/2018-03-20/12_7193_cli...,Парижское соглашение сегодня в России у всех н...,Искажение буквы и смысла договорных текстов св...,"[парижский, соглашение, сегодня, россия, уста,...","[парижский, соглашение, трудность, перевод]"
996,"[юрий коваль, проза, стихи, юбилей, троцкий, м...",А Троцкого убили кирпичом,https://amp.ng.ru/?p=http://www.ng.ru/ng_exlib...,"Юрий Коваль – писатель нужный, чрезвычайно нео...",Завтра исполняется 80 лет со дня рождения писа...,"[юрий, коваль, писатель, нужный, чрезвычайно, ...","[троцкий, убить, кирпич]"
997,"[кинофестиваль, дух огня, хантымансийск, кино]",Кино в городе сверкающего снега,http://www.ng.ru/cinematograph/2018-03-12/100_...,"«Дух огня», международный смотр кинематографич...","Ханты-Мансийский фестиваль ""Дух огня"" присудил...","[дух, огонь, международный, смотр, кинематогра...","[кино, город, сверкать, снег]"


Посчитаем бейзлайн для тестового фрагмента данных

In [0]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [0]:
test_data['content_norm'] = test_data['content'].apply(normalize)
test_data['title_norm'] = test_data['title'].apply(normalize)

In [0]:
from itertools import combinations
#Функция для работы с графами позаимствована в семинарской тетрадке
def get_kws(text, top=10, window_size=10, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        s = np.sum(m[i])
        if not s:
            continue
        m[i] /= s
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        
        if go_random:
            n = np.random.choice(len(vocab))
        
        
        ### 
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    if np.any(matrix[n]):
        next_n = np.random.choice(range(rang), p=matrix[n])
    else:
        next_n = np.random.choice(range(rang))
    return next_n

In [0]:
keywords_rw = test_data['content_norm'].apply(lambda x: get_kws(x, 10, 10))
evaluate(test_data['keywords'], keywords_rw)

Precision -  0.08
Recall -  0.15
F1 -  0.1
Jaccard -  0.05


In [0]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

In [0]:
def keyword_with_stemming(lemmas):
  stem_to_word = {}
  for lemma in lemmas:
    parsed_word = morph.parse(lemma)[0]
    if 'NOUN' in parsed_word.tag:
        stem_to_word[stemmer.stem(lemma)]=lemma
  stems = [stemmer.stem(lemma) for lemma in lemmas]
  chosen = [stem_to_word[stem] for stem in stems if stem in stem_to_word]
  kws = get_kws(chosen, 10, 10)
  return kws

In [0]:
keywords_rw = test_data['content_norm'].apply(lambda x: keyword_with_stemming(x))

In [0]:
evaluate(test_data['keywords'], keywords_rw)

Precision -  0.11
Recall -  0.21
F1 -  0.14
Jaccard -  0.08


Метрики удалось повысить

Идея 2. 
Сделаем граф направленным. Попробуем два направления и выберем, какое работает лучше.

In [0]:
def get_kws_directed1(text, top=10, window_size=10, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            #m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        s = np.sum(m[i])
        if not s:
            continue
        m[i] /= s
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        
        if go_random:
            n = np.random.choice(len(vocab))
        
        
        ### 
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    if np.any(matrix[n]):
        next_n = np.random.choice(range(rang), p=matrix[n])
    else:
        next_n = np.random.choice(range(rang))
    return next_n

In [0]:
def get_kws_directed2(text, top=10, window_size=10, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            #m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        s = np.sum(m[i])
        if not s:
            continue
        m[i] /= s
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        
        if go_random:
            n = np.random.choice(len(vocab))
        
        
        ### 
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    if np.any(matrix[n]):
        next_n = np.random.choice(range(rang), p=matrix[n])
    else:
        next_n = np.random.choice(range(rang))
    return next_n

In [0]:
keywords_rw = test_data['content_norm'].apply(lambda x: get_kws_directed1(x, 10, 10))
evaluate(test_data['keywords'], keywords_rw)

Precision -  0.05
Recall -  0.11
F1 -  0.07
Jaccard -  0.04


In [0]:
keywords_rw = test_data['content_norm'].apply(lambda x: get_kws_directed2(x, 10, 10))
evaluate(test_data['keywords'], keywords_rw)

Precision -  0.06
Recall -  0.13
F1 -  0.08
Jaccard -  0.04


Становится только хуже

Проверим, поможет ли в комбинации со стемингом

In [0]:
def keyword_with_stemming_directed(lemmas):
  stem_to_word = {}
  for lemma in lemmas:
    parsed_word = morph.parse(lemma)[0]
    if 'NOUN' in parsed_word.tag:
        stem_to_word[stemmer.stem(lemma)]=lemma
  stems = [stemmer.stem(lemma) for lemma in lemmas]
  chosen = [stem_to_word[stem] for stem in stems if stem in stem_to_word]
  kws = get_kws_directed2(chosen, 10, 10)
  return kws

In [0]:
keywords_rw = test_data['content_norm'].apply(lambda x: keyword_with_stemming_directed(x))
evaluate(test_data['keywords'], keywords_rw)

Precision -  0.09
Recall -  0.18
F1 -  0.11
Jaccard -  0.06


Стеминг без однонаправленности помогает лучше

Идея 3. Попробуем повысить значимость слов из заголовка, прибавив к в соответствующих строках и столбцах n при составлении матрицы

In [0]:
def get_kws_title_using_title(text, title, top=10, window_size=10, random_p=0.1, k=3):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    for word in title: #повышаем вероятность перехода в слова из заголовка
      if word in word2id:
        j = word2id[word]
        for i in range(len(vocab)):
          m[i][j] += k

    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        s = np.sum(m[i])
        if not s:
            continue
        m[i] /= s
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        
        if go_random:
            n = np.random.choice(len(vocab))
        
        
        ### 
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    if np.any(matrix[n]):
        next_n = np.random.choice(range(rang), p=matrix[n])
    else:
        next_n = np.random.choice(range(rang))
        return next_n

In [0]:
context_norm = list(test_data['content_norm'])
title_norm = list(test_data['title_norm'])
key_words = []
for i in range(len(context_norm)):
  key_words.append(get_kws_title_using_title(text=context_norm[i], title=title_norm[i])) #k=3
keywords_rw = pd.DataFrame({'text':context_norm, 'kw':key_words}) 

evaluate(test_data['keywords'], keywords_rw['kw'])

Precision -  0.08
Recall -  0.16
F1 -  0.1
Jaccard -  0.06


In [0]:
context_norm = list(test_data['content_norm'])
title_norm = list(test_data['title_norm'])
key_words = []
for i in range(len(context_norm)):
  key_words.append(get_kws_title_using_title(text=context_norm[i], title=title_norm[i], k=1))
keywords_rw = pd.DataFrame({'text':context_norm, 'kw':key_words})

evaluate(test_data['keywords'], keywords_rw['kw'])

Precision -  0.08
Recall -  0.17
F1 -  0.1
Jaccard -  0.06


In [0]:
context_norm = list(test_data['content_norm'])
title_norm = list(test_data['title_norm'])
key_words = []
for i in range(len(context_norm)):
  key_words.append(get_kws_title_using_title(text=context_norm[i], title=title_norm[i], k=5))
keywords_rw = pd.DataFrame({'text':context_norm, 'kw':key_words})

evaluate(test_data['keywords'], keywords_rw['kw'])

Precision -  0.08
Recall -  0.17
F1 -  0.1
Jaccard -  0.06


Ничего хорошего не получается. Попробуем сделать k зависимым от длины текста

In [0]:
context_norm = list(test_data['content_norm'])
title_norm = list(test_data['title_norm'])
key_words = []
for i in range(len(context_norm)):
  k = len(context_norm[i])//30
  key_words.append(get_kws_title_using_title(text=context_norm[i], title=title_norm[i], k=k))
keywords_rw = pd.DataFrame({'text':context_norm, 'kw':key_words})

evaluate(test_data['keywords'], keywords_rw['kw'])

Precision -  0.08
Recall -  0.16
F1 -  0.1
Jaccard -  0.06


In [0]:
context_norm = list(test_data['content_norm'])
title_norm = list(test_data['title_norm'])
key_words = []
for i in range(len(context_norm)):
  k = len(context_norm[i])//5
  key_words.append(get_kws_title_using_title(text=context_norm[i], title=title_norm[i], k=k))
keywords_rw = pd.DataFrame({'text':context_norm, 'kw':key_words})

evaluate(test_data['keywords'], keywords_rw['kw'])

Precision -  0.08
Recall -  0.16
F1 -  0.1
Jaccard -  0.06


Всё равно ничего хорошего не получается

Идея 4. Оставим от текста только существительные

In [0]:
def nouns(lemmas):
  nouns = []
  for lemma in lemmas:
    parsed_word = morph.parse(lemma)[0]
    if 'NOUN' in parsed_word.tag:
      nouns.append(lemma)
  return nouns

In [0]:
test_data['content_nouns'] = test_data['content_norm'].apply(nouns)

In [0]:
keywords_rw = test_data['content_nouns'].apply(lambda x: get_kws(x, window_size=5))
evaluate(test_data['keywords'], keywords_rw)

Precision -  0.13
Recall -  0.13
F1 -  0.12
Jaccard -  0.07


Стало лучше, так как не возвращаются слова "не тех" частей речи, как и при использовании функции со стемингом (первой из предложенных в этой тетрадке)

Идея 5. Возьмём самые частотные существительные без использования графов

In [0]:
keywords_rw = test_data['content_nouns'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)])
evaluate(test_data['keywords'], keywords_rw)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


Precision такой же, как при использовании графов, остальные метрики выше

### Идея 6. Попробуем взять n самых частотных биграм и 10-n самых частотных существительных

In [0]:
def bigrams(text):
  tokens = [word.strip(punct) for word in text.lower().split()]
  bigrams = []
  for i in range(1, len(tokens)):
    bigram = tokens[i-1]+' '+tokens[i]
    bigrams.append(bigram)
  return(bigrams)

In [0]:
test_data['bigrams'] = test_data['content'].apply(bigrams)

Просматриванием ответов удалось прийти к выводу, что в основном ключевыми являются биграмы следующих типов:
- существительное + существительное, первое в начальной форме, второе - зависимое (управление) - в той форме, в какой в тексте
- прилагательное + существительное, существительное - в начальной форме, прилагательное с ним согласуется

In [0]:
def prepare_bigrams(bigrams):
  prepared_bigrams = []
  for bigram in bigrams:
    try: #возможно исключение при попытке получить род существительного pluralia tantum
      [token1, token2] = bigram.split()
      parsed_word1 = morph.parse(token1)[0]
      parsed_word2 = morph.parse(token2)[0]
      if 'NOUN' in parsed_word1.tag and 'NOUN' in parsed_word2.tag:
        prepared_bigram = parsed_word1.normal_form + ' ' + token2
        prepared_bigrams.append(prepared_bigram)
      elif 'NOUN' in parsed_word2.tag and 'ADJF' in parsed_word1.tag:
        gender = parsed_word1.tag.gender
        prepared_token1 = parsed_word1.inflect({gender, 'nomn', 'sing'})
        prepared_token2 = parsed_word2.normal_form
        prepared_bigram = prepared_token1 + ' ' + prepared_token2
        prepared_bigrams.append(prepared_bigram)
    except:
      pass 
    return prepared_bigrams

In [0]:
nouns = list(test_data['content_nouns'])
prepared_bigrams = [prepare_bigrams(bigrams) for bigrams in text_as_bigrams]
keys = []
for i in range(len(nouns)):
    keys.append(bigrams_and_nouns(prepared_bigrams[i], nouns[i]))

In [0]:
key_values = []
for key in keys:
    key_text = [res[0] for res in key]
    key_values.append(key_text)
test_data['keys_bigrams'] = key_values
evaluate(test_data['keywords'], test_data['keys_bigrams'])

Считала в другой тетрадке, получила результат:
Precision -  0.11
Recall -  0.21
F1 -  0.13
Jaccard -  0.08
Не лучше, чем при использовании только существительных, но ощутимо дольше

Идея 7
Применим tfidf к спискам существительных

In [0]:
test_data['nouns_str'] = test_data['content_nouns'].apply(' '.join)

In [0]:
tfidf = TfidfVectorizer(min_df=5)

In [16]:
tfidf.fit(test_data['nouns_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(test_data['nouns_str'])
keywords = []
for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])
evaluate(test_data['keywords'], keywords)

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09


Очень похоже на использование Counter, но проигрывает по recall

(сюда я вставлю результаты по всем файлам, когда они досчитаются)